<a href="https://colab.research.google.com/github/habbes/learn-nanogpt/blob/main/learn_nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on the lecture "Let's build GPT: from scratch, in code, spelled out" by Andrej Karpathy: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-22 04:44:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-07-22 04:44:38 (26.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()


In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# Let's look at the first 1000 characters

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# We're building a character-level language model for simplicity
# We map each character to an integer (index of the character in our vocabulary in this case)
# Encoding a piece of text will return a sequence of integers
# This is a naive tokenizer for learning purposes
# Sample "real-world" tokenizers:
# - https://github.com/google/sentencepiece
# - https://github.com/openai/tiktoken
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# Let's now encode the entire text data set and store it into torch.Tensor
import torch # PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will look like this to the GPT

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Let's split the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
print(train_data.shape)
print(val_data.shape)

torch.Size([1003854])
torch.Size([111540])


In [11]:
# When training, we want pass the entire dataset to the transformer at once, that would be computationally prohibitive.
# We'll break data into chunks. We sample random chunks from the training set and train chunks at a time.
# Chunks have a max length. Let's call it block size, aka context length

In [12]:
# Sample block size = 8
block_size = 8
# Let's look at the first chunk in the training data
train_data[:block_size + 1] # we add the (block_size+1)th element so that we can train [:block_size] context to predict [block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
# This block provides many training examples, 8 to be precise. Each subsequence is an example, and the value after the subsequence a prediction:

# context                          -> target output
# [18]                             -> 47
# [18, 47]                         -> 56
# [18, 47, 56]                     -> 57
# [18, 47, 56, 57]                 -> 58
# [18, 47, 56, 57, 58]             -> 1
# [18, 47, 56, 57, 58, 1]          -> 15
# [18, 47, 56, 57, 58, 1, 15]      -> 47
# [18, 47, 56, 57, 58, 1, 15, 47]  -> 58

# This helps the transformer generalize over a wide range of context lengths

In [15]:
# Let's the concept in the previous block into code
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [20]:
# Batching allows process independent sequences in parallel. This is done for efficiency and leverage the parallelism of the GPU
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

# the batch_size represents the batch dimension
# the block_size represents the time dimension (assuming a sequence of tokens that make up a sentence or ordered chronologically)

def get_batch(split):
  # generate a small batch of data inputs x and target y
  data = train_data if split == 'train' else val_data
  # generates a batch_size-sized sequence of random indexes (index ranginge from 0 to N - block_size)
  # Each index is an offset in the data to the start of a batch of size block_size.
  # So the last possible index would be N - block_size - 1 (the -1 so that we can have on item after the context block as expected output)
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  # The generated indices allow us to select a batch of random training blocks from the dataset
  # create a batch of training context blocks
  # torch.stack creates rows of data where each row corresponds to one of the input lists
  x = torch.stack([data[i:i + block_size] for i in ix])
  # create a batch of training predictions
  y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
  return x, y

# get a random batch from training data
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("----")

# The 4x8 batch array encapsulates 32 independent training examples
# that we can see by expanding the loop below
for b in range(batch_size): # batch dimension
  for t in range(block_size): #time dimension
    context = xb[b, :t + 1]
    target = yb[b, t]
    print(f"when input context is {context.tolist()} the target is: { target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input context is [24] the target is: 43
when input context is [24, 43] the target is: 58
when input context is [24, 43, 58] the target is: 5
when input context is [24, 43, 58, 5] the target is: 57
when input context is [24, 43, 58, 5, 57] the target is: 1
when input context is [24, 43, 58, 5, 57, 1] the target is: 46
when input context is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when input context is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
when input context is [44] the target is: 53
when input context is [44, 53] the target is: 56
when input context is [44, 53, 56] the target 

In [21]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
# Let's start with the simplest neural network for language models: bigram language model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    # this is a lookup table that has vocab_size entries/rows and each
    # entry is a vector of size vocab_size
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):
    # idx and targets are both (B,T) tensor of integers (batch of context blocks and corresponding batch of targets)
    # remember each encoded token in the input data as index into the vocab table, and therefore an index into
    # the embedding table as well
    # For each input integer in the (B,T) input batch, find its corresponding vector from the embedding table
    # So for row in B and col in T, we'll have a vector of size vocab_size (C)
    # So for all the input tokens, the lookup will return a tensor of size (B, T, C) where each input token is mapped to its embedding vector
    logits = self.token_embedding_table(idx) # (B,T,C) C refers to channel, vocab size in this case

    return logits

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)